In [ ]:
!pip install earthengine-api ipywidgets

In [1]:
from map_machine import mapper
from map_machine.ui.cli import parse_arguments
# from maploc.get_bev import get_osm_data, draw_bev
from map_machine.geometry.boundary_box import BoundaryBox
from pathlib import Path
from map_machine.scheme import Scheme
from map_machine.map_configuration import MapConfiguration
from xml.etree import ElementTree as ET
from map_machine.osm.osm_reader import OSMData
import colour
import matplotlib.pyplot as plt
import numpy as np
from map_machine.geometry.flinger import MercatorFlinger
# from mapper.data.get_bev import get_osm_data, draw_bev, process_img, get_bev_from_bbox
# import numpy as np
from map_machine.mapper import Map
from map_machine.constructor import Constructor
from map_machine.pictogram.icon import ShapeExtractor
from map_machine.workspace import workspace
import svgwrite
import cairosvg

import ee, requests, io
from PIL import Image
from ipywidgets import interact, FloatSlider

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-tony200303zt')

In [18]:
bbox_str = "-79.92367,40.46017,-79.92170,40.46121"

In [19]:
bbox = BoundaryBox.from_text(bbox_str)
bbox_ee = [float(x) for x in bbox_str.split(",")]

In [20]:
def get_osm_data(bbox):
    ENDPOINT = "http://overpass.kumi.systems/api/map"
    bbox = bbox

    response = requests.get(f"{ENDPOINT}?bbox={bbox}")
    xml_str = response.content.decode("utf-8")

    return xml_str

In [22]:
scheme = Scheme.from_file(Path("mapstyle.yaml"))
conf = MapConfiguration(scheme, show_credit=False)
tree = ET.fromstring(get_osm_data(bbox_str))
osm_data = OSMData()
osm_data.parse_osm(tree, parse_nodes=True, 
                    parse_relations=True, parse_ways=True) 

In [23]:
def draw_bev(bbox: BoundaryBox, osm_data: OSMData,
             configuration: MapConfiguration, meters_per_pixel: float):
    """Rasterize OSM data as a BEV image"""
    lat = bbox.center()[0]
    # Equation rearranged from https://wiki.openstreetmap.org/wiki/Zoom_levels
    # To get zoom level given meters_per_pixel
    z = np.log2(np.abs(osm_data.equator_length*np.cos(np.deg2rad(lat))/meters_per_pixel/256))
    flinger = MercatorFlinger(bbox, z, osm_data.equator_length)

    size = flinger.size
    svg: svgwrite.Drawing = svgwrite.Drawing(None, size) # None since we are not saving an svg file

    icon_extractor: ShapeExtractor = ShapeExtractor(
        workspace.ICONS_PATH, workspace.ICONS_CONFIG_PATH
    )
    constructor: Constructor = Constructor(
        osm_data=osm_data,
        flinger=flinger,
        extractor=icon_extractor,
        configuration=configuration,
    )
    constructor.construct()
    map_: Map = Map(flinger=flinger, svg=svg, configuration=configuration)
    imgs = []

    map_.draw(constructor)

    png_byte_string = cairosvg.svg2png(bytestring=svg.tostring(), 
                                            output_width=size[0], 
                                            output_height=size[1]) # convert svg to png
    img = Image.open(io.BytesIO(png_byte_string))

    return img

In [24]:
def get_satellite_from_bbox(bbox, output_fp, num_pixels, heading):
    region = ee.Geometry.Rectangle(bbox, proj="EPSG:4326", geodesic=False)
    # Load a satellite image collection, filter it by date and region, then select the first image
    image = ee.ImageCollection('USDA/NAIP/DOQQ') \
        .filterBounds(region) \
        .filterDate('2022-01-01', '2022-12-31') \
        .sort('CLOUDY_PIXEL_PERCENTAGE') \
        .first().select(['R', 'G', 'B'])

    # Reproject the image to a common projection (e.g., EPSG:4326)
    image = image.reproject(crs='EPSG:4326', scale=0.5)

    # Get the image URL
    url = image.getThumbURL({'min': 0, 'max': 255, 'region': region.getInfo()['coordinates']})

    # Download the image to your desktop
    response = requests.get(url)
    img = Image.open(io.BytesIO(response.content))

    return img

In [25]:
sat_img = get_satellite_from_bbox(bbox_ee, "satellite.png", 1000, 0)

In [26]:
map_img = draw_bev(bbox=bbox, osm_data=osm_data, configuration=conf, meters_per_pixel=0.05)
map_img = map_img.resize(sat_img.size)

In [27]:
def show_images(alpha):
    plt.imshow(sat_img)
    plt.imshow(map_img, alpha=alpha)
    plt.show()

In [28]:
interact(show_images, alpha=FloatSlider(min=0., max=1., step=0.1))

interactive(children=(FloatSlider(value=0.0, description='alpha', max=1.0), Output()), _dom_classes=('widget-i…

<function __main__.show_images(alpha)>